In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
# --- CONFIG ---
img_size = 224
batch_size = 32
epochs = 10
learning_rate = 1e-4
dataset_path = "D:/22.11.5308/SMT 6/Data Mining/Dataset/CT-Scan Lung Cancer"

# --- Data Generators ---
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_gen.class_indices)

Found 2888 images belonging to 3 classes.
Found 721 images belonging to 3 classes.


In [4]:
# --- Model Setup ---
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# --- Training ---
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs
)

Epoch 1/10
91/91 [==============================] - 165s 2s/step - loss: 0.5406 - accuracy: 0.7621 - val_loss: 0.5547 - val_accuracy: 0.7420
Epoch 2/10
91/91 [==============================] - 169s 2s/step - loss: 0.3048 - accuracy: 0.8864 - val_loss: 0.6846 - val_accuracy: 0.7157
Epoch 3/10
91/91 [==============================] - 155s 2s/step - loss: 0.2108 - accuracy: 0.9318 - val_loss: 0.3538 - val_accuracy: 0.8460
Epoch 4/10
91/91 [==============================] - 155s 2s/step - loss: 0.1473 - accuracy: 0.9581 - val_loss: 0.4977 - val_accuracy: 0.8211
Epoch 5/10
91/91 [==============================] - 178s 2s/step - loss: 0.1139 - accuracy: 0.9706 - val_loss: 0.3699 - val_accuracy: 0.8544
Epoch 6/10
91/91 [==============================] - 165s 2s/step - loss: 0.0884 - accuracy: 0.9820 - val_loss: 0.4517 - val_accuracy: 0.8336
Epoch 7/10
91/91 [==============================] - 225s 2s/step - loss: 0.0759 - accuracy: 0.9799 - val_loss: 0.2901 - val_accuracy: 0.8932
Epoch 8/10
91

In [6]:
# --- Evaluation ---
loss, accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

23/23 [==============================] - 32s 1s/step - loss: 0.4139 - accuracy: 0.8779
Validation Loss: 0.4139
Validation Accuracy: 0.8779
